In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]


In [4]:
spark = SparkSession.builder.appName("prepare").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/26 04:25:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
empDF = spark.createDataFrame(emp,empColumns)

In [6]:
# empDF.show()

In [7]:
empDF.printSchema()

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [8]:
# empDF.show(truncate=False)

In [9]:
dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]

In [10]:
deptColumns = ["dept_name","dept_id"]

In [11]:
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)

In [12]:
deptDF.show(truncate=False)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [14]:
empDF.alias("a").join(deptDF,a.emp_dept_id == deptDF.dept_id, how="inner").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [13]:
df2 = empDF.union(empDF)

In [23]:

from pyspark.sql.functions import col

ImportError: cannot import name 'analyzer' from 'pyspark.sql' (/usr/local/lib/python3.11/site-packages/pyspark/sql/__init__.py)

In [24]:
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", False)

In [25]:
df2.alias("a").join(empDF, empDF.emp_id == col("a.superior_emp_id"), how="leftanti").show()
         

+------+-----+---------------+-----------+-----------+------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|     1|Smith|             -1|       2018|         10|     M|  3000|
|     1|Smith|             -1|       2018|         10|     M|  3000|
+------+-----+---------------+-----------+-----------+------+------+



In [15]:
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, how = "fullouter").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  NULL|    NULL|           NULL|       NULL|       NULL|  NULL|  NULL|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     NULL|   NULL|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [42]:
empDF.join(deptDF, deptDF.dept_id == empDF.emp_dept_id, how="leftanti").show()

+------+-----+---------------+-----------+-----------+------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|     6|Brown|              2|       2010|         50|      |    -1|
+------+-----+---------------+-----------+-----------+------+------+



In [17]:
from pyspark.sql.functions import col

In [18]:
emp3 = empDF.alias("emp1").join(empDF.alias("emp2"), col("emp1.superior_emp_id") == col("emp2.emp_id"), "inner")

In [19]:
emp3.select(col("emp1.emp_id"), col("emp2.name")).show()

+------+-----+
|emp_id| name|
+------+-----+
|     2|Smith|
|     3|Smith|
|     4| Rose|
|     5| Rose|
|     6| Rose|
+------+-----+



In [20]:
emp3.show()

+------+--------+---------------+-----------+-----------+------+------+------+-----+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+------+-----+---------------+-----------+-----------+------+------+
|     2|    Rose|              1|       2010|         20|     M|  4000|     1|Smith|             -1|       2018|         10|     M|  3000|
|     3|Williams|              1|       2010|         10|     M|  1000|     1|Smith|             -1|       2018|         10|     M|  3000|
|     4|   Jones|              2|       2005|         10|     F|  2000|     2| Rose|              1|       2010|         20|     M|  4000|
|     5|   Brown|              2|       2010|         40|      |    -1|     2| Rose|              1|       2010|         20|     M|  4000|
|     6|   Brown|          

In [21]:
# Using Spark SQL

empDF.createOrReplaceTempView("EMP")

In [27]:
deptDF.createOrReplaceTempView("DEPT")

joinDF = spark.sql("select * from EMP e, DEPT d where e.emp_dept_id = d.dept_id")

In [52]:
joinDF.distinct(col("gender")).show()

TypeError: DataFrame.distinct() takes 1 positional argument but 2 were given

In [46]:
joinDF2 = spark.sql("select * from EMP e INNER JOIN DEPT d ON e.emp_dept_id == d.dept_id") 

In [47]:
joinDF3 = spark.sql("select * from EMP e INNER JOIN DEPT d ON e.emp_dept_id == d.dept_id")

In [48]:
joinDF.join(joinDF2, joinDF2.superior_emp_id == joinDF.emp_id, how="outer")

AnalysisException: Column emp_id#658L are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.